In [1]:
import findspark
findspark.init('/usr/local/spark/')

In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql.functions import split, explode

In [5]:
spark = SparkSession.builder\
            .appName('Streaming WC').master('local[*]')\
            .getOrCreate()

In [6]:
wcDf = spark.readStream.format('socket')\
            .option('host','localhost')\
            .option('port','9999')\
            .load()

In [7]:
splitExpDf = wcDf.withColumn('value',explode(split('value', ' ')))
cntDf = splitExpDf.groupBy('value').count()

In [8]:
query = cntDf \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("strmg") \
    .start()

from IPython.display import display, clear_output
from time import sleep

while True:
    clear_output(wait=True)
    display(query.status)
    display(spark.sql('SELECT * FROM strmg').show())
    sleep(3)

{'isDataAvailable': False,
 'isTriggerActive': False,
 'message': 'Waiting for data to arrive'}

+------+-----+
| value|count|
+------+-----+
|  saif|    5|
|  juhi|    1|
|  arif|    1|
|saachi|    2|
|      |    1|
+------+-----+



None

KeyboardInterrupt: 